# LinearModel

In [ ]:
import SparseArrays: sparse
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb")
@nbinclude("EnsembleInputs.ipynb")
@nbinclude("SuppressImplicit.ipynb");

## Training

In [ ]:
function save_linear_model(alphas, implicit, outdir)
    set_logging_outdir(outdir)
    _, β = regress(alphas, implicit)
    @info "alphas: $alphas"
    @info "coefficients: $β"
    write_params(Dict("β" => β, "alphas" => alphas), outdir)

    splits = []
    @showprogress for split in all_nontraining_raw_splits
        push!(splits, read_alpha(alphas, split, implicit))
    end
    splits = reduce(cat, splits)
    preds = sparse(splits.user, splits.item, splits.rating)
    write_alpha(preds, [], implicit, outdir; log_test_split = true)
end;

In [ ]:
save_linear_model(explicit_raw_alphas, false, "LinearExplicit");

In [ ]:
save_linear_model(implicit_raw_alphas, true, "LinearImplicit");